# ML DEFRA Data Preparation for Air Quality Prediction

This notebook prepares DEFRA data for machine learning models - mirrors LAQN ml_prep exactly.

## What this notebook does

1. Loads cleaned DEFRA data from the optimised folder.

   ```bash
   ├── optimised/
   │   ├── 2023measurements/
   │   │   ├── Station_Name/
   │   │   │   ├── NO2__2023_01.csv
   │   │   │   ├── PM10__2023_01.csv
   │   │   │   └── ...
   │   ├── 2024measurements/
   │   └── 2025measurements/
   ```

2. Combines all measurements into a single dataset.
3. Creates temporal features (hour, day, month).
4. Creates sequences for ML training.

## DEFRA CSV Structure

Each CSV file contains:
```
timestamp, value, timeseries_id, station_name, pollutant_name, pollutant_std, latitude, longitude
```

## Output path:

Data will be saved to: `data/defra/ml_prep/`

In [1]:
# Standard imports same as LAQN
import pandas as pd
import numpy as np
import os
from pathlib import Path

# Save section
import joblib

# Visualisation
import matplotlib.pyplot as plt

# Preprocessing libraries
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

## File Paths

- Usual drill, adding paths under this cell for organisation.

In [16]:
# DEFRA prep file paths
base_dir = Path.cwd().parent.parent / "data" / "defra"
project_root = Path.cwd() / "defra_ml_prep.ipynb"

# DEFRA optimised data path
optimised_path = base_dir / "optimised"

# Output paths
output_path = base_dir / "ml_prep"
output_path.mkdir(parents=True, exist_ok=True)

# Visualisation output path
visualisation_path = output_path / "visualisation"
visualisation_path.mkdir(parents=True, exist_ok=True)

print(f"Base directory: {base_dir}")
print(f"Data path: {optimised_path}")
print(f"Output path: {output_path}")
print(f"Path exists: {optimised_path.exists()}")

Base directory: /Users/burdzhuchaglayan/Desktop/data science projects/air-pollution-levels/data/defra
Data path: /Users/burdzhuchaglayan/Desktop/data science projects/air-pollution-levels/data/defra/optimised
Output path: /Users/burdzhuchaglayan/Desktop/data science projects/air-pollution-levels/data/defra/ml_prep
Path exists: True


## 1) Load DEFRA data

DEFRA structure is nested: `optimised/YEARmeasurements/Station_Name/Pollutant__YYYY_MM.csv`

CSV columns: `timestamp, value, timeseries_id, station_name, pollutant_name, pollutant_std, latitude, longitude`

We'll map these to match LAQN column names for consistency:
- `timestamp` → `@MeasurementDateGMT`
- `value` → `@Value`
- `pollutant_std` → `SpeciesCode`
- `station_name` → `SiteCode` (with spaces replaced)

In [17]:
def load_defra_data(optimised_path):
    """
    Function to load the optimased data files from the defra dataset.

            optimised_path: path for data/defra/optimased directory.

    """
    optimised_path = Path(optimised_path)
    all_files = []
    file_count = 0
    
    # Get year measurement folders (2023measurements, 2024measurements, etc.)
    year_folders = sorted([f for f in optimised_path.iterdir() 
                          if f.is_dir() and 'measurements' in f.name])
    
    print(f"Found {len(year_folders)} year folders")
    
    for year_dir in year_folders:
        year_file_count = 0
        
        # Get station folders inside each year
        station_folders = sorted([f for f in year_dir.iterdir() if f.is_dir()])
        
        for station_dir in station_folders:
            # Get all CSV files in station folder
            for csv_file in station_dir.glob("*.csv"):
                try:
                    df = pd.read_csv(csv_file)
                    all_files.append(df)
                    file_count += 1
                    year_file_count += 1
                except Exception as e:
                    print(f"Error reading {csv_file.name}: {e}")
        
        print(f"  Loaded {year_dir.name}: {year_file_count} files")
    
    # Combine all dataframes
    if not all_files:
        raise ValueError(f"No CSV files found in {optimised_path}")
    
    combined_df = pd.concat(all_files, ignore_index=True)
    
    print(f"\n" + "="*40)
    print(f"Total files loaded: {file_count}")
    print(f"Total rows: {len(combined_df):,}")
    print(f"Columns: {list(combined_df.columns)}")
    
    return combined_df

In [18]:
# Load all defra data
df_raw = load_defra_data(optimised_path)
df_raw.head()

Found 3 year folders
  Loaded 2023measurements: 1431 files
  Loaded 2024measurements: 1193 files
  Loaded 2025measurements: 939 files

Total files loaded: 3563
Total rows: 2,525,991
Columns: ['timestamp', 'value', 'timeseries_id', 'station_name', 'pollutant_name', 'pollutant_std', 'latitude', 'longitude']


,timestamp,value,timeseries_id,station_name,pollutant_name,pollutant_std,latitude,longitude
0,2023-09-01 00:00:00,26.966,4566,Borehamwood Meadow Park,Nitrogen oxides,NOx,51.661229,-0.27055
1,2023-09-01 01:00:00,27.349,4566,Borehamwood Meadow Park,Nitrogen oxides,NOx,51.661229,-0.27055
2,2023-09-01 02:00:00,22.567,4566,Borehamwood Meadow Park,Nitrogen oxides,NOx,51.661229,-0.27055
3,2023-09-01 04:00:00,17.021,4566,Borehamwood Meadow Park,Nitrogen oxides,NOx,51.661229,-0.27055
4,2023-09-01 05:00:00,23.141,4566,Borehamwood Meadow Park,Nitrogen oxides,NOx,51.661229,-0.27055
